In [66]:
'''
Definizione delle funzioni di supporto
'''

def remove_number_some_punctuation_marks(row):
    '''
    remove_number:   rimuove i caratteri numerici e il carattere '"' dalla stringa e la trasforma in lower-case
    '''
    lowercase = row.lower()
    lowercase = lowercase.replace("--", " ")
    res = ""
    
    for char in lowercase:
        if not ('0' <= char <= '9' or char == '"'):
            res += char

    return res

def remove_number_punctuation_marks(row):
    '''
    remove_number:   rimuove i caratteri numerici dalla stringa e i segni di punteggiatura 
                     e la trasforma in lower-case
    '''
    
    lowercase = row.lower()
    lowercase = lowercase.replace("--", " ")
    res = ""
    
    for char in lowercase:
        if 'a' <= char <= 'z' or char == ' ' or char == '-' or char == "'":
            res += char

    return res

In [116]:
'''
Definizione delle funzioni che calcolano le metriche per la Authorship Attribution
'''

def word_counter(RDD):
    '''
    word_counter:   è la funzione che dato un RDD conta il numero di volte che compare una parola.
                    Inoltre per risparmiare operazioni ritorna anche l'ATTRIBUTO VOCABULARY_SIZE
    '''
    word_counter = (RDD.flatMap(lambda x: x)
                .map(lambda x: (x,1))
                .reduceByKey(lambda a,b: a+b)
                .sortBy(lambda x: -x[1])
               )
    return word_counter.collect(), len(word_counter.collect())

def text_length_in_words(word_count):
    '''
    text_length_in_words:   calcola il numero di parole del testo usando word_counter
    '''
    s = 0
    for couple in word_count:
        s += couple[1]
    return s

def ratio_V_T(voc_size, text_len):
    '''
    ratio_V_T:   calcola il rapporto fra la dimensione del vocabolario e il numero totale di parole
                 nel testo
    '''
    return voc_size / text_len

def hentropy(word_count,text_len):
    import math
    '''
    hentropy:   funzione che calcola l'entropia usando l'Entropia di Gibbs. L'unica differenza
                è la mancanza della costante di Boltzmann
    '''
    s = 0
    
    for couple in word_count:
        s += (couple[1]/text_len) * math.log2(couple[1]/text_len)
    
    return -s

def maximum_sentence_length(sentence_collection):
    '''
    maximum_sentence_length:   funzione che calcola la lunghezza (in numero di parole) della frase
                               più lunga
    '''
    _max = float("-inf")
    
    for sentence in sentence_collection:
        if len(sentence) > _max:
            _max = len(sentence)
    return _max

def minimum_sentence_length(sentence_collection):
    '''
    minimum_sentence_length:   funzione che calcola la lunghezza (in numero di parole) della frase
                               più corta
    '''
    _min = float("inf")
    
    for sentence in sentence_collection:
        if len(sentence) < _min and len(sentence) > 0:
            _min = len(sentence)
    return _min

def average_sentence_length(sentence_collection):
    '''
    average_sentence_length:   funzione che calcola la lunghezza media(in numero di parole)
                               delle frasi del testo
    '''
    _sum = 0
    
    for sentence in sentence_collection:
        _sum += len(sentence)
    return _sum / len(sentence_collection)


def prob_of_the_most_freq_sentence_len(sentence_collection):
    '''
    prob_of_the_most_freq_sentence_len:   funzione che calcola la probabilità
            della lunghezza della frase più frequente
    '''
    
    # coppia (lunghezza_frase, numero_frasi_con_"lunghezza_frase")
    d = dict()
    
    for sentence in sentence_collection:
        try:
            d[len(sentence)] += 1
        except:
            d[len(sentence)] = 1
    
    _max_freq = 0
    
    for key in d:
        if _max_freq < d[key]:
            _max_freq = d[key]
    
    return _max_freq / len(sentence_collection)

def prob_distr_of_30_most_common_words(word_count):
    '''
    prob_distr_of_30_most_common_words: funzione che ritorna una lista di probabilità
                                        delle 30 parole di uso più comune
    '''    
    res = []
    _sum = 0
    
    for couple in word_count:
        _sum += couple[1]
    
    # word_count è ordinato
    for i in range(0, 30):
        res.append((word_count[i][0], word_count[i][1] / _sum))
        
    return res

def prob_distr_of_30_most_common_words_except_art_prep(word_count, art_prep):
    '''
    prob_distr_of_30_most_common_words_except_art_prep: funzione che ritorna una lista di probabilità
                                                        delle 30 parole di uso più comune escludendo articoli
                                                        e preposizioni
    '''
    
    res = []
    _sum = 0
    
    for couple in word_count:
        if couple[0] not in art_prep:
            _sum += couple[1]
    
    # word_count è ordinato
    i = 0
    counter = 0
    
    for couple in word_count:
        if couple[0] not in art_prep:
            res.append((word_count[i][0], word_count[i][1] / _sum))
            counter += 1
        
        i += 1

        if counter == 29:
            break

    return res



def prob_distr_of_sentence_length(sentence_collection):
    '''
    prob_distr_of_sentence_length:   funzione che calcola la distribuzione di probabilità
                                     della lunghezza della frasi
    '''    
    d = dict()
    
    for sentence in sentence_collection:
        try:
            d[len(sentence)] += 1
        except:
            d[len(sentence)] = 1
        
    
    for key in d:
        d[key] /= len(sentence_collection)
    
    return d


def prob_of_The(word_count):
    '''
    prob_of_The: funzione che ritorna la probabilità di "the" 
    '''
    _sum = 0
    value = 0
    for couple in word_count:
        if couple[0] == "the":
            value = couple[1]
        _sum += couple[1]
    
    return value / _sum

def prob_of_comma(sentences_collection):
    '''
    prob_of_The: funzione che ritorna la probabilità che capiti una virgola
    '''
    
    _sum = 0
    value = 0
    for sentence in sentences_collection:
        for word in sentence:         
            if "," in word:
                value += 1
            _sum += 1

    return value / _sum

def prob_of_the_most_common_word(word_count):
    '''
    prob_of_the_most_common_word: funzione che ritorna la probabilità della parola più
                                  comune escudendo "the" e "and"
    '''
    # distribuzione di probabilità
    pd = prob_distr_of_30_most_common_words(word_count)
    
    for couple in pd:
        if couple[0] != "and" and couple[0] != "the":
            return couple

def prob_of_the_most_common_word_except_art_prep(word_count, art_prep):
    '''
    prob_of_the_most_common_word_except_art_prep: funzione che ritorna la probabilità della parola più
                                                  comune escudendo articoli e preposizioni
    '''
    
    # distribuzione di probabilità
    pd = prob_distr_of_30_most_common_words_except_art_prep(word_count, art_prep)
    
    return pd[0]

def MCW_except_the(word_count):
    '''
    MCW_except_the: funzione che ritorna la parola più comune a eccezione di "the"
    '''
    for couple in word_count:
        if couple[0] != "the":
            return couple[0]

def MCW_except_art_prep(word_count, art_prep):
    '''
    MCW_except_the: funzione che ritorna la parola più comune a eccezione di "the"
    '''
    for couple in word_count:
        if couple[0] not in art_prep:
            return couple[0]
        
def avg_distance_consec_appear_MCW(data):
    '''
    avg_distance_consec_appear_MCW: funzione che calcola la distanza media di due parole MCW consecutive.
                                    MCW indica la parola di uso più comune escludendo la parola "the"
    '''
    MCW = MCW_except_the(word_counter(data)[0])
    
    ds = []
    ds.append(1) # inizializziamo la lista delle distanze
    
    for line in data.collect():
        for word in line:
            if MCW == word:
                ds.append(1)
            else:
                ds[len(ds) - 1] += 1
    _avg = 0
    
    for d in ds:
        _avg += d
    
    return _avg / len(ds)

def min_distance_consec_appear_MCW(data):
    MCW = MCW_except_the(word_counter(data)[0])
    
    ds = []
    ds.append(1) # inizializziamo la lista delle distanze
    
    for line in data.collect():
        for word in line:
            if MCW == word:
                ds.append(1)
            else:
                ds[len(ds) - 1] += 1

    return min(ds)

def max_distance_consec_appear_MCW(data):
    MCW = MCW_except_the(word_counter(data)[0])
    
    ds = []
    ds.append(1) # inizializziamo la lista delle distanze
    
    for line in data.collect():
        for word in line:
            if MCW == word:
                ds.append(1)
            else:
                ds[len(ds) - 1] += 1

    return max(ds)

def avg_distance_consec_appear_MCW_except_art_prep(data, art_prep):
    '''
    avg_distance_consec_appear_MCW_except_art_prep: funzione che calcola la distanza media di due parole MCW consecutive.
                                                    MCW indica la parola di uso più comune escludendo articoli e preposizioni
    '''
    MCW = MCW_except_art_prep(word_counter(data)[0], art_prep)
    
    ds = []
    ds.append(1) # inizializziamo la lista delle distanze
    
    for line in data.collect():
        for word in line:
            if MCW == word:
                ds.append(1)
            else:
                ds[len(ds) - 1] += 1
    _avg = 0
    
    for d in ds:
        _avg += d
    
    return _avg / len(ds)

def min_distance_consec_appear_MCW_except_art_prep(data, art_prep):
    
    MCW = MCW_except_art_prep(word_counter(data)[0], art_prep)
    
    ds = []
    ds.append(1) # inizializziamo la lista delle distanze
    
    for line in data.collect():
        for word in line:
            if MCW == word:
                ds.append(1)
            else:
                ds[len(ds) - 1] += 1

    return min(ds)

def max_distance_consec_appear_MCW_except_art_prep(data, art_prep):
    MCW = MCW_except_art_prep(word_counter(data)[0], art_prep)
    
    ds = []
    ds.append(1) # inizializziamo la lista delle distanze
    
    for line in data.collect():
        for word in line:
            if MCW == word:
                ds.append(1)
            else:
                ds[len(ds) - 1] += 1

    return max(ds)


def avg_distance_consec_appear_of_The(data):
    '''
    avg_distance_consec_appear_MCW: funzione che calcola la distanza media di due parole "the" consecutive.
                                    MCW indica la parola di uso più comune escludendo la parola "the"
    '''    
    ds = []
    ds.append(1) # inizializziamo la lista delle distanze
    
    for line in data.collect():
        for word in line:
            if "the" == word:
                ds.append(1)
            else:
                ds[len(ds) - 1] += 1
    _avg = 0
    
    for d in ds:
        _avg += d
    
    return _avg / len(ds)

def min_distance_consec_appear_of_The(data):
    '''
    min_distance_consec_appear_of_The: funzione che calcola la distanza minima di due parole "the" consecutive.
                                       MCW indica la parola di uso più comune escludendo la parola "the"
    ''' 
    ds = []
    ds.append(1) # inizializziamo la lista delle distanze
    
    for line in data.collect():
        for word in line:
            if "the" == word:
                ds.append(1)
            else:
                ds[len(ds) - 1] += 1

    return min(ds)

def max_distance_consec_appear_of_The(data):
    '''
    max_distance_consec_appear_of_The: funzione che calcola la distanza massima di due parole "the" consecutive.
                                       MCW indica la parola di uso più comune escludendo la parola "the"
    ''' 
    ds = []
    ds.append(1) # inizializziamo la lista delle distanze
    
    for line in data.collect():
        for word in line:
            if "the" == word:
                ds.append(1)
            else:
                ds[len(ds) - 1] += 1

    return max(ds)


def avg_distance_consec_appear_of_comma(senteces_collection):
    '''
    avg_distance_consec_appear_of_comma: funzione che calcola la distanza media di due consecutive virgole.
    '''    
    ds = []
    ds.append(1) # inizializziamo la lista delle distanze
    
    for line in senteces_collection:
        for word in line:
            if "," in word:
                ds.append(1)
            else:
                ds[len(ds) - 1] += 1
    _avg = 0
    
    for d in ds:
        _avg += d
    
    return _avg / len(ds)

def min_distance_consec_appear_of_comma(senteces_collection):
    '''
    min_distance_consec_appear_of_comma: funzione che calcola la distanza minima di due consecutive virgole.
    ''' 
    ds = []
    ds.append(1) # inizializziamo la lista delle distanze
    
    for sentence in senteces_collection:
        for word in sentence:
            if "," in word:
                ds.append(1)
            else:
                ds[len(ds) - 1] += 1

    return min(ds)

def max_distance_consec_appear_of_comma(senteces_collection):
    '''
    max_distance_consec_appear_of_comma: funzione che calcola la distanza massima di due consecutive virgole.
    ''' 
    ds = []
    ds.append(1) # inizializziamo la lista delle distanze
    
    for sentence in senteces_collection:
        for word in sentence:
            if "," in word:
                ds.append(1)
            else:
                ds[len(ds) - 1] += 1

    return max(ds)

In [100]:
# CARICAMENTO PREPOSIZIONI E ARTICOLI

raw_art_prep = sc.textFile("preposizioni_e_articoli.txt")

# RIMUOVIAMO NUMERI + SEGNI DI PUNTEGGIATURA
art_prep = raw_art_prep.filter(bool)

# POSIAMO I DATI NELLA CACHE
art_prep.persist()

PythonRDD[184] at RDD at PythonRDD.scala:53

In [101]:
# CARICAMENTO DATASET

rawData = sc.textFile("datasets/Anthony Trollope___The O'Conors of Castle Conor from Tales from all Countries.txt")

# RIMUOVIAMO NUMERI + SEGNI DI PUNTEGGIATURA
data = (rawData.filter(bool)                    # rimuoviamo le stringhe vuote
        .map(remove_number_punctuation_marks)
        .map(lambda x : ' '.join(x.split()))    # rimuoviamo diversi spazi bianchi con uno
        .map(lambda row : row.split(" "))
       )

# POSIAMO I DATI NELLA CACHE
data.persist()

PythonRDD[187] at RDD at PythonRDD.scala:53

In [96]:
# word_counter, e ATTRIBUTO vocabulary_size
word_count, voc_size = word_counter(data)

In [119]:
# SOMMA LE FREQUENZE DENTRO word_count PER TROVARE L'ATTRIBUTO text_length_in_words
text_len = text_length_in_words(word_count)
print(text_len)

7600


In [118]:
# ATTRIBUTO definito rapporto vocabulary_size / text_length_in_words
ratio_V_T(voc_size, text_len)

0.21039473684210527

In [73]:
# ENTROPIA misura formalmente la quantità di disordine
hentropy(word_count,text_len)

8.639377320109679

In [103]:
# linee del file in input
lines = (rawData.filter(bool)
            .map(remove_number_some_punctuation_marks)
            .map(lambda x : ' '.join(x.split()))
            )

# METTO TUTTO IL TESTO IN UNA STRINGA UNICA
text = ''
for s in lines.collect():
    text += s + ' '

text = text.replace("?", ".")
text = text.replace("!", ".")

# LISTA DI STRINGHE. ESSE SONO FRASI
sentences = text.split(".")

# LISTA DI LISTE CHE CONTENGONO LE PAROLE DI UNA FRASE
senteces_collection = sc.parallelize(sentences).map(lambda x: x.split()).collect()

#print(senteces_collection)

# 81
print(maximum_sentence_length(senteces_collection))
# 1
print(minimum_sentence_length(senteces_collection))
# 17.473563218390805
print(average_sentence_length(senteces_collection))

84
1
14.833333333333334


In [104]:
prob_of_the_most_freq_sentence_len(senteces_collection)

0.06395348837209303

In [76]:
prob_distr_of_30_most_common_words(word_count)

[('the', 0.04513157894736842),
 ('i', 0.03605263157894737),
 ('and', 0.0325),
 ('to', 0.02368421052631579),
 ('of', 0.02131578947368421),
 ('a', 0.019736842105263157),
 ('my', 0.01631578947368421),
 ('that', 0.016052631578947367),
 ('in', 0.015921052631578947),
 ('was', 0.015394736842105263),
 ('said', 0.012236842105263157),
 ('he', 0.01118421052631579),
 ('as', 0.011052631578947368),
 ('but', 0.009736842105263158),
 ('at', 0.009605263157894737),
 ('you', 0.008947368421052631),
 ('for', 0.008815789473684211),
 ('it', 0.008552631578947369),
 ('me', 0.008421052631578947),
 ('had', 0.007894736842105263),
 ('with', 0.007236842105263158),
 ('not', 0.00631578947368421),
 ('all', 0.0061842105263157894),
 ("o'conor", 0.0061842105263157894),
 ('his', 0.005789473684210527),
 ('were', 0.005789473684210527),
 ('on', 0.005394736842105263),
 ('so', 0.005),
 ('there', 0.004736842105263158),
 ('we', 0.004736842105263158)]

In [77]:
prob_of_the_most_common_word(word_count)

('i', 0.03605263157894737)

In [78]:
avg_distance_consec_appear_MCW(data)

27.64

In [79]:
min_distance_consec_appear_MCW(data)

2

In [80]:
max_distance_consec_appear_MCW(data)

176

In [81]:
avg_distance_consec_appear_of_The(data)

22.09593023255814

In [82]:
min_distance_consec_appear_of_The(data)

1

In [83]:
max_distance_consec_appear_of_The(data)

115

In [84]:
prob_of_The(word_count)

0.04513157894736842

In [85]:
avg_distance_consec_appear_of_comma(senteces_collection)

14.7495183044316

In [86]:
min_distance_consec_appear_of_comma(senteces_collection)

1

In [87]:
max_distance_consec_appear_of_comma(senteces_collection)

92

In [88]:
prob_of_comma(senteces_collection)

0.06767703161745492

In [109]:
prob_distr_of_sentence_length(senteces_collection)

{7: 0.06395348837209303,
 35: 0.011627906976744186,
 20: 0.029069767441860465,
 12: 0.040697674418604654,
 17: 0.04263565891472868,
 26: 0.013565891472868217,
 41: 0.005813953488372093,
 23: 0.01937984496124031,
 6: 0.03682170542635659,
 18: 0.02131782945736434,
 61: 0.001937984496124031,
 2: 0.0562015503875969,
 28: 0.013565891472868217,
 38: 0.005813953488372093,
 25: 0.023255813953488372,
 39: 0.001937984496124031,
 59: 0.005813953488372093,
 54: 0.001937984496124031,
 30: 0.009689922480620155,
 15: 0.03875968992248062,
 48: 0.001937984496124031,
 10: 0.046511627906976744,
 8: 0.03875968992248062,
 24: 0.015503875968992248,
 14: 0.02131782945736434,
 4: 0.031007751937984496,
 1: 0.03875968992248062,
 22: 0.023255813953488372,
 3: 0.03875968992248062,
 5: 0.03488372093023256,
 9: 0.046511627906976744,
 11: 0.03488372093023256,
 29: 0.01937984496124031,
 27: 0.007751937984496124,
 63: 0.001937984496124031,
 32: 0.003875968992248062,
 46: 0.003875968992248062,
 16: 0.03875968992248062,

In [111]:
avg_distance_consec_appear_MCW_except_art_prep(data, art_prep.collect())

27.141843971631207

In [113]:
min_distance_consec_appear_MCW_except_art_prep(data, art_prep.collect())

2

In [112]:
max_distance_consec_appear_MCW_except_art_prep(data, art_prep.collect())

177

In [115]:
prob_distr_of_30_most_common_words_except_art_prep(word_count, art_prep.collect())

[('i', 0.04608139926000673),
 ('and', 0.04154053144971409),
 ('my', 0.020854355869492094),
 ('that', 0.020517995290951902),
 ('was', 0.019677093844601413),
 ('said', 0.01564076690211907),
 ('he', 0.014295324587958291),
 ('you', 0.011436259670366633),
 ('it', 0.01093171880255634),
 ('me', 0.010763538513286243),
 ('had', 0.010090817356205853),
 ('not', 0.008072653884964682),
 ('all', 0.007904473595694584),
 ("o'conor", 0.007904473595694584),
 ('his', 0.007399932727884292),
 ('were', 0.007399932727884292),
 ('so', 0.006390850992263707),
 ('there', 0.006054490413723511),
 ('we', 0.006054490413723511),
 ('mr', 0.005886310124453414),
 ('this', 0.005886310124453414),
 ('larry', 0.005886310124453414),
 ('which', 0.005886310124453414),
 ('be', 0.005886310124453414),
 ('is', 0.005718129835183316),
 ('then', 0.0053817692566431215),
 ('have', 0.005213588967373024),
 ('jack', 0.005213588967373024),
 ('they', 0.004877228388832829)]

In [117]:
prob_of_the_most_common_word_except_art_prep(word_count, art_prep.collect())

('i', 0.04608139926000673)